# Финальный проект

Самый высокий precision@5 удалось получить:

    - упростив функцию prefilter_items - берем топ n самых популярных товаров и убираем остальные фильтры;
    - если взять 5000 самых популярных товаров;
    - при bm25_weight с параметрами K1=3, B=0.3;
    - модель первого уровня - own recommendtions + top-popular;
    - при отборе 100 кандидатов моделью первого уровня;
    - при добавлении новых признаков в модель второго уровня;
    - LightAutoML улучшает качество рекомендаций (по сравнению с LightGBM моделью);
    - эксперименты с различными весами в user-item матрице не принесли интересных результатов.

# План

    1. Двухуровневая модель для покупателей, когда нам известна информация о предыдущих покупках
        a) Выбираем модель первого уровня
        b) Модель второго уровня
            - LightGBM модель
            - LightAutoML модель
     2. Модель для новых покупателей, о покупках которых у нас нет информации
     3. Объединение рекомендаций, полученных на этапе 1 и 2
     4. Валидация

In [1]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модели второго уровня
from lightgbm import LGBMClassifier

from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task
from lightautoml.tasks.common_metric import mean_quantile_error

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [2]:
data = pd.read_csv('../retail_train.csv')
item_features = pd.read_csv('../product.csv')
user_features = pd.read_csv('../hh_demographic.csv')

In [3]:
# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

In [4]:
# Схема обучения и валидации
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [5]:
# упростим функцию prefilter_items и возьмем топ n самых популярных товаров, убрав остальные фильтры

n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

/Users/nadejdaberzina/Documents/Geekbrains_New/Chetvertaya_chetvert/recsys_final_project/final_project/src/utils.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 83685 to 5001


/Users/nadejdaberzina/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


## 1. Двухуровневая модель для покупателей, когда нам известна информация об их предыдущих покупках

Начнем с юзеров, по которым у нас уже есть информацию из data train.

Для новых юзеров построим модель на следующем этапе.

### a) Выбираем модель первого уровня

In [6]:
#обучаем модель первого уровня на data train level 1
recommender = MainRecommender(data_train_lvl_1)

In [7]:
# создаем dataframe, куда будем сохранять кандидатов, сгенерированных моделей первого уровня
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']


# отбираем юзеров, по которым у нас есть информация из data train
users_lvl_1 = pd.DataFrame(data_val_lvl_1['user_id'].unique())
users_lvl_1.columns = ['user_id']

train_users = data_train_lvl_1['user_id'].unique()
users_lvl_1 = users_lvl_1[users_lvl_1['user_id'].isin(train_users)]


# включаем в результирующий dataframe только отобранных юзеров
result_lvl_1 = users_lvl_1.merge(result_lvl_1, on=['user_id'], how='left')

In [8]:
# генерируем кандидатов
k = 100

result_lvl_1['als'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=k))
result_lvl_1['own'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=k))
result_lvl_1['similar_items'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_similar_items_recommendation(x, N=k))
result_lvl_1['similar_users'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_similar_users_recommendation(x, N=k))

In [9]:
def calc_metrics(data, metric, k=100):
    
    for column in data.columns[2:]:
        
        yield column, data.apply(lambda row: metric(row[column], row['actual'], k=k), axis=1).mean()

In [10]:
sorted(calc_metrics(result_lvl_1, recall_at_k), key=lambda recall_at_k: recall_at_k[1], reverse=True)

[('own', 0.21068430151403514),
 ('als', 0.15145427295287625),
 ('similar_items', 0.09212067843290002),
 ('similar_users', 0.08442621972412903)]

In [11]:
sorted(calc_metrics(result_lvl_1, precision_at_k, k=5), key=lambda precision_at_k: precision_at_k[1], reverse=True)

[('own', 0.380213655364611),
 ('als', 0.2045517882025048),
 ('similar_users', 0.12679981421272488),
 ('similar_items', 0.09391546679052427)]

Own recommendtions + top-popular дают наилучший recall при отборе 100 кандидатов, а также наилучший presion@5.

### b) Модель второго уровня

### - LightGBM модель

Сначала протестируем различные параметры на LightGBM модели, чтобы расчеты занимали меньше времени.
Следующим этапом применим LightAutoML модель, сохранив параметры, при которых LightGBM покажет наилучший результат.

##### Обучим модель второго уровня на data_train_lvl_2

In [12]:
# отберем юзеров, по которым у нас есть информация из data train
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

In [13]:
# генерируем кандидатов
users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=100))

In [14]:
# создаем dataframe для модели второго уровня с флагом купил / не купил
s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)

targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)

In [15]:
# добавим имеющиеся данные по пользователям и товарам
targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left')

In [16]:
# генерируем новые признаки
# средний чек
average_receipt = data_train_lvl_2.groupby(['user_id', 'basket_id'])['sales_value'].sum().reset_index()
average_receipt = average_receipt.groupby('user_id')['sales_value'].mean().reset_index()
average_receipt.columns=['user_id', 'average_receipt']
targets_lvl_2 = targets_lvl_2.merge(average_receipt, how='left', on='user_id')

In [17]:
# генерируем новые признаки
# кол-во покупок в каждой категории
purchases_per_category = data_train_lvl_2.merge(item_features, on='item_id', how='left')
purchases_per_category = purchases_per_category.groupby(['user_id', 'commodity_desc'])['quantity'].sum().reset_index() # кол-во покупок в каждой категории
purchases_per_category.columns=['user_id', 'commodity_desc', 'purchases_per_category']
targets_lvl_2 = targets_lvl_2.merge(purchases_per_category, how='left', on=['user_id', 'commodity_desc'])

In [18]:
# генерируем новые признаки
# кол-во покупок в каждой под-категории
purchases_per_sub_category = data_train_lvl_2.merge(item_features, on='item_id', how='left')
purchases_per_sub_category = purchases_per_sub_category.groupby(['user_id', 'sub_commodity_desc'])['quantity'].sum().reset_index() # кол-во покупок в каждой категории
purchases_per_sub_category.columns=['user_id', 'sub_commodity_desc', 'purchases_per_sub_category']
targets_lvl_2 = targets_lvl_2.merge(purchases_per_sub_category, how='left', on=['user_id', 'sub_commodity_desc'])

In [19]:
# генерируем новые признаки
# покупки в каждой под-категории в денежном эквиваленте
value_per_sub_category = data_train_lvl_2.merge(item_features, on='item_id', how='left')
value_per_sub_category = value_per_sub_category.groupby(['user_id', 'sub_commodity_desc'])['sales_value'].sum().reset_index() # кол-во покупок в каждой категории
value_per_sub_category.columns=['user_id', 'sub_commodity_desc', 'value_per_sub_category']
targets_lvl_2 = targets_lvl_2.merge(value_per_sub_category, how='left', on=['user_id', 'sub_commodity_desc'])

In [20]:
# генерируем новые признаки
# среднее количество покупок в неделю для каждого товара
purchases_per_week = data_train_lvl_2.merge(item_features, on='item_id', how='left')
purchases_per_week = purchases_per_week.groupby(['item_id', 'week_no'])['quantity'].sum().reset_index()
purchases_per_week = purchases_per_week.groupby('item_id')['quantity'].mean().reset_index()
purchases_per_week.columns=['item_id', 'purchases_per_week']
targets_lvl_2 = targets_lvl_2.merge(purchases_per_week, how='left', on='item_id')

In [21]:
# готовим X_train и y_train
X_train = targets_lvl_2.drop('target', axis=1)
y_train = targets_lvl_2[['target']]

In [22]:
cat_feats = X_train.columns[2:15].tolist()

In [23]:
X_train[cat_feats] = X_train[cat_feats].astype('category')

In [24]:
lgb = LGBMClassifier(objective='binary', max_depth=7, categorical_column=cat_feats)
lgb.fit(X_train, y_train)

/Users/nadejdaberzina/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/nadejdaberzina/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/nadejdaberzina/opt/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:863: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


LGBMClassifier(boosting_type='gbdt',
               categorical_column=['manufacturer', 'department', 'brand',
                                   'commodity_desc', 'sub_commodity_desc',
                                   'curr_size_of_product', 'age_desc',
                                   'marital_status_code', 'income_desc',
                                   'homeowner_desc', 'hh_comp_desc',
                                   'household_size_desc', 'kid_category_desc'],
               class_weight=None, colsample_bytree=1.0, importance_type='split',
               learning_rate=0.1, max_depth=7, min_child_samples=20,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
               n_jobs=-1, num_leaves=31, objective='binary', random_state=None,
               reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

##### Проверяем качество модели на data_val_lvl_2

In [25]:
# отберем юзеров, по которым у нас есть информация из data train
users_lvl_2_val = pd.DataFrame(data_val_lvl_2['user_id'].unique())
users_lvl_2_val.columns = ['user_id']

train_users_lvl_2 = users_lvl_2['user_id'].unique()
users_lvl_2_val = users_lvl_2_val[users_lvl_2_val['user_id'].isin(train_users_lvl_2)]


# создаем dataframe, куда будем сохранять рекомендации
result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_2.columns=['user_id', 'actual']

result_lvl_2 = result_lvl_2[result_lvl_2['user_id'].isin(targets_lvl_2.user_id.unique())]

result_lvl_2 = users_lvl_2_val.merge(result_lvl_2, how='left', on='user_id')

In [26]:
# оценим качество для модели первого уровня
result_lvl_2['own'] = result_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=5))

In [27]:
# генерируем кандидатов
users_lvl_2_val['candidates'] = users_lvl_2_val['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=100))

In [28]:
# создаем dataframe для модели второго уровня с флагом купил / не купил
s = users_lvl_2_val.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

users_lvl_2_val = users_lvl_2_val.drop('candidates', axis=1).join(s)

targets_lvl_2_val = data_val_lvl_2[['user_id', 'item_id']].copy()

targets_lvl_2_val = users_lvl_2_val.merge(targets_lvl_2_val, on=['user_id', 'item_id'], how='left')

In [29]:
# добавим имеющиеся данные по пользователям и товарам
targets_lvl_2_val = targets_lvl_2_val.merge(item_features, on='item_id', how='left')
targets_lvl_2_val = targets_lvl_2_val.merge(user_features, on='user_id', how='left')

In [30]:
# генерируем новые признаки
# средний чек
average_receipt = data_val_lvl_2.groupby(['user_id', 'basket_id'])['sales_value'].sum().reset_index()
average_receipt = average_receipt.groupby('user_id')['sales_value'].mean().reset_index()
average_receipt.columns=['user_id', 'average_receipt']
targets_lvl_2_val = targets_lvl_2_val.merge(average_receipt, on='user_id', how='left')

In [31]:
# генерируем новые признаки
# кол-во покупок в каждой категории
purchases_per_category = data_val_lvl_2.merge(item_features, on='item_id', how='left')
purchases_per_category = purchases_per_category.groupby(['user_id', 'commodity_desc'])['quantity'].sum().reset_index() # кол-во покупок в каждой категории
purchases_per_category.columns=['user_id','commodity_desc', 'purchases_per_category']
targets_lvl_2_val = targets_lvl_2_val.merge(purchases_per_category, how='left', on=['user_id', 'commodity_desc'])

In [32]:
# генерируем новые признаки
# кол-во покупок в каждой под-категории
purchases_per_sub_category = data_val_lvl_2.merge(item_features, on='item_id', how='left')
purchases_per_sub_category = purchases_per_sub_category.groupby(['user_id', 'sub_commodity_desc'])['quantity'].sum().reset_index() # кол-во покупок в каждой категории
purchases_per_sub_category.columns=['user_id','sub_commodity_desc', 'purchases_per_sub_category']
targets_lvl_2_val = targets_lvl_2_val.merge(purchases_per_sub_category, how='left', on=['user_id', 'sub_commodity_desc'])

In [33]:
# генерируем новые признаки
# покупки в каждой под-категории в денежном эквиваленте
value_per_sub_category = data_val_lvl_2.merge(item_features, on='item_id', how='left')
value_per_sub_category = value_per_sub_category.groupby(['user_id', 'sub_commodity_desc'])['sales_value'].sum().reset_index() # кол-во покупок в каждой категории
value_per_sub_category.columns=['user_id','sub_commodity_desc', 'value_per_sub_category']
targets_lvl_2_val = targets_lvl_2_val.merge(value_per_sub_category, how='left', on=['user_id', 'sub_commodity_desc'])

In [34]:
# генерируем новые признаки
# среднее количество покупок в неделю для каждого товара
purchases_per_week = data_val_lvl_2.merge(item_features, on='item_id', how='left')
purchases_per_week = purchases_per_week.groupby(['item_id', 'week_no'])['quantity'].sum().reset_index()
purchases_per_week = purchases_per_week.groupby('item_id')['quantity'].mean().reset_index()
purchases_per_week.columns=['item_id', 'purchases_per_week']
targets_lvl_2_val = targets_lvl_2_val.merge(purchases_per_week, how='left', on='item_id')

In [35]:
# готовим данные
cat_feats = targets_lvl_2_val.columns[2:15].tolist()
targets_lvl_2_val[cat_feats] = targets_lvl_2_val[cat_feats].astype('category')

In [36]:
val_preds = lgb.predict_proba(targets_lvl_2_val)

In [37]:
# создаем dataframe с предсказаниями и считаем целевую метрику для модели первого уровня и модели 

val_preds = val_preds[:, 1]

result_val = targets_lvl_2_val
result_val['preds'] = val_preds

result_val = result_val.groupby(['user_id', 'item_id'])['preds'].mean().reset_index()
result_val = result_val.groupby('user_id').apply(lambda x: x.sort_values('preds', ascending=False).head(5).item_id.tolist())
result_val = result_lvl_2.merge(result_val.rename('lgbm'), how='left', on='user_id')

sorted(calc_metrics(result_val, precision_at_k, k=5), key=lambda precision_at_k: precision_at_k[1], reverse=True)

[('lgbm', 0.618485639686683), ('own', 0.3427676240208894)]

### - LightAutoML модель

In [38]:
train_data = targets_lvl_2

In [39]:
TASK = Task('reg', loss='mse', metric='mse', greater_is_better=False)
TIMEOUT = 300000
N_THREADS = 4
N_FOLDS = 5
RANDOM_STATE = 42
TARGET_NAME = 'target'
TEST_SIZE=0.2

In [40]:
roles = {'target': TARGET_NAME, 'drop': ['user_id, user_id']}

In [41]:
automl_model = TabularAutoML(task=TASK,
                            timeout=TIMEOUT,
                            cpu_limit = N_THREADS,
                            gpu_ids='all',
                            reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE},
                             
                            general_params={'use_algos': [ ['lgb_tuned', 'cb_tuned', 'cb', 'lgb'], ['lgb_tuned', 'cb'] ]},
                             
                            tuning_params={'max_tuning_iter': 10},
                      )

In [42]:
automl_pred = automl_model.fit_predict(train_data, roles = roles)

INFO:optuna.storages._in_memory:A new study created in memory with name: no-name-4074d9a9-4868-4550-a5b1-90c1c12f1011
INFO:optuna.study.study:Trial 0 finished with value: -0.036848281857779434 and parameters: {'feature_fraction': 0.6872700594236812, 'num_leaves': 244}. Best is trial 0 with value: -0.036848281857779434.
INFO:optuna.study.study:Trial 1 finished with value: -0.03700059369165765 and parameters: {'feature_fraction': 0.8659969709057025, 'num_leaves': 159}. Best is trial 0 with value: -0.036848281857779434.
INFO:optuna.study.study:Trial 2 finished with value: -0.03691918204138139 and parameters: {'feature_fraction': 0.5780093202212182, 'num_leaves': 53}. Best is trial 0 with value: -0.036848281857779434.
INFO:optuna.study.study:Trial 3 finished with value: -0.03686356827586462 and parameters: {'feature_fraction': 0.5290418060840998, 'num_leaves': 223}. Best is trial 0 with value: -0.036848281857779434.
INFO:optuna.study.study:Trial 4 finished with value: -0.03666716858713051 

In [43]:
automl_pred_val = automl_model.predict(targets_lvl_2_val)

In [44]:
result = automl_pred_val.data
targets_lvl_2_val['preds_automl'] = result
targets_lvl_2_val['preds_automl'] = abs(targets_lvl_2_val['preds_automl'])

targets_lvl_2_val = targets_lvl_2_val.groupby(['user_id', 'item_id'])['preds_automl'].mean().reset_index()

result_val_automl = targets_lvl_2_val.groupby('user_id').apply(lambda x: x.sort_values('preds_automl', ascending=False).head(5).item_id.tolist())

result_val_automl = result_lvl_2.merge(result_val_automl.rename('automl'), how='left', on='user_id')

sorted(calc_metrics(result_val_automl, precision_at_k, k=5), key=lambda precision_at_k: precision_at_k[1], reverse=True)

[('automl', 0.6432375979112254), ('own', 0.3427676240208894)]

## 2. Модель для новых покупателей, о покупках которых у нас нет информации

In [45]:
recs = pd.DataFrame(data_val_lvl_2['user_id'].unique())
recs.columns = ['user_id']

old_users = result_val['user_id'].unique()
new_users = recs[~recs['user_id'].isin(old_users)]

In [46]:
def popularity_recommendation(data, n=5):
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    return recs.tolist()

In [47]:
popular_recs = popularity_recommendation(data_train_lvl_1, n=6)
popular_recs = popular_recs[1:]

new_users['recs'] = new_users['user_id'].apply(lambda x: popular_recs)

/Users/nadejdaberzina/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


## 3. Объединение рекомендаций, полученных на этапе 1 и 2

In [48]:
result_val_new = result_val_automl[['user_id', 'automl']]
result_val_new.rename(columns={'automl': 'recs'}, inplace=True)

/Users/nadejdaberzina/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [49]:
recommendations = pd.concat([result_val_new, new_users])

In [50]:
result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_2.columns=['user_id', 'actual']

In [51]:
recs = result_lvl_2.merge(recommendations, on='user_id', how='left')

In [52]:
# рассчитаем precision@5
sorted(calc_metrics(recs, precision_at_k, k=5), key=lambda precision_at_k: precision_at_k[1], reverse=True)

[('recs', 0.6068560235063635)]

In [53]:
recs = recs.drop(columns=['actual'])

In [54]:
recs.to_csv('predictions.csv', index=False)

## 4. Валидация

Сгенерируем новые тренировочный и тестовый наборы, чтобы еще раз проверить качество модели.

Исключим из наших данных последние три недели, и разобьем данные на тренировочный и тестовый наборы.

In [55]:
data = data[data['week_no'] <= (data['week_no'].max() - 3)]

In [56]:
# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 

val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [57]:
# берем только топ n самых популярных товаров

n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

/Users/nadejdaberzina/Documents/Geekbrains_New/Chetvertaya_chetvert/recsys_final_project/final_project/src/utils.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 82059 to 5001


/Users/nadejdaberzina/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


##### Oбучаем модель первого уровня

In [58]:
#обучаем модель первого уровня на data train level 1
recommender = MainRecommender(data_train_lvl_1)

##### Генерируем кандидатов

In [59]:
# отберем юзеров, по которым у нас есть информация из data train
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

In [60]:
# генерируем кандидатов
users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=100))

##### Oбучаем модель второго уровня

In [61]:
# создаем dataframe для модели второго уровня с флагом купил / не купил
s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)

targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)

In [62]:
# добавим имеющиеся данные по пользователям и товарам
targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left')

In [63]:
# генерируем новые признаки
# средний чек
average_receipt = data_train_lvl_2.groupby(['user_id', 'basket_id'])['sales_value'].sum().reset_index()
average_receipt = average_receipt.groupby('user_id')['sales_value'].mean().reset_index()
average_receipt.columns=['user_id', 'average_receipt']
targets_lvl_2 = targets_lvl_2.merge(average_receipt, how='left', on='user_id')

In [64]:
# генерируем новые признаки
# кол-во покупок в каждой категории
purchases_per_category.columns=['user_id','commodity_desc', 'purchases_per_category']
purchases_per_category = data_train_lvl_2.merge(item_features, on='item_id', how='left')
purchases_per_category = purchases_per_category.groupby(['user_id', 'commodity_desc'])['quantity'].sum().reset_index()
purchases_per_category.columns=['user_id','commodity_desc', 'purchases_per_category']
targets_lvl_2 = targets_lvl_2.merge(purchases_per_category, how='left', on=['user_id', 'commodity_desc'])

In [65]:
# генерируем новые признаки
# кол-во покупок в каждой под-категории
purchases_per_sub_category = data_train_lvl_2.merge(item_features, on='item_id', how='left')
purchases_per_sub_category = purchases_per_sub_category.groupby(['user_id', 'sub_commodity_desc'])['quantity'].sum().reset_index()
purchases_per_sub_category.columns=['user_id', 'sub_commodity_desc', 'purchases_per_sub_category']
targets_lvl_2 = targets_lvl_2.merge(purchases_per_sub_category, how='left', on=['user_id', 'sub_commodity_desc'])

In [66]:
# генерируем новые признаки
# покупки в каждой под-категории в денежном эквиваленте
value_per_sub_category = data_train_lvl_2.merge(item_features, on='item_id', how='left')
value_per_sub_category = value_per_sub_category.groupby(['user_id', 'sub_commodity_desc'])['sales_value'].sum().reset_index()
value_per_sub_category.columns=['user_id', 'sub_commodity_desc', 'value_per_sub_category']
targets_lvl_2 = targets_lvl_2.merge(value_per_sub_category, how='left', on=['user_id', 'sub_commodity_desc'])

In [67]:
# генерируем новые признаки
# среднее количество покупок в неделю для каждого товара
purchases_per_week = data_train_lvl_2.merge(item_features, on='item_id', how='left')
purchases_per_week = purchases_per_week.groupby(['item_id', 'week_no'])['quantity'].sum().reset_index()
purchases_per_week = purchases_per_week.groupby('item_id')['quantity'].mean().reset_index()
purchases_per_week.columns=['item_id', 'purchases_per_week']
targets_lvl_2 = targets_lvl_2.merge(purchases_per_week, how='left', on='item_id')

In [68]:
# готовим X_train и y_train
X_train = targets_lvl_2.drop('target', axis=1)
y_train = targets_lvl_2[['target']]

In [69]:
cat_feats = X_train.columns[2:15].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

In [70]:
# обучаем LGBM
lgb = LGBMClassifier(objective='binary', max_depth=7, categorical_column=cat_feats)
lgb.fit(X_train, y_train)

/Users/nadejdaberzina/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/nadejdaberzina/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/nadejdaberzina/opt/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:863: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


LGBMClassifier(boosting_type='gbdt',
               categorical_column=['manufacturer', 'department', 'brand',
                                   'commodity_desc', 'sub_commodity_desc',
                                   'curr_size_of_product', 'age_desc',
                                   'marital_status_code', 'income_desc',
                                   'homeowner_desc', 'hh_comp_desc',
                                   'household_size_desc', 'kid_category_desc'],
               class_weight=None, colsample_bytree=1.0, importance_type='split',
               learning_rate=0.1, max_depth=7, min_child_samples=20,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
               n_jobs=-1, num_leaves=31, objective='binary', random_state=None,
               reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [71]:
# обучаем LightAutoML
train_data = targets_lvl_2

In [72]:
TASK = Task('reg', loss='mse', metric='mse', greater_is_better=False)
TIMEOUT = 300000
N_THREADS = 4
N_FOLDS = 5
RANDOM_STATE = 42
TARGET_NAME = 'target'
TEST_SIZE=0.2

In [73]:
roles = {'target': TARGET_NAME, 'drop': ['user_id, user_id']}

In [74]:
automl_model = TabularAutoML(task=TASK,
                            timeout=TIMEOUT,
                            cpu_limit = N_THREADS,
                            gpu_ids='all',
                            reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE},
                             
                            general_params={'use_algos': [ ['lgb_tuned', 'cb_tuned', 'cb', 'lgb'], ['lgb_tuned', 'cb'] ]},
                             
                            tuning_params={'max_tuning_iter': 10},
                      )

In [75]:
automl_pred = automl_model.fit_predict(train_data, roles = roles)

INFO:optuna.storages._in_memory:A new study created in memory with name: no-name-c0562d6b-3db4-465a-81ea-b13a4c6e622d
INFO:optuna.study.study:Trial 0 finished with value: -0.03713762915464922 and parameters: {'feature_fraction': 0.6872700594236812, 'num_leaves': 244}. Best is trial 0 with value: -0.03713762915464922.
INFO:optuna.study.study:Trial 1 finished with value: -0.03715186985415576 and parameters: {'feature_fraction': 0.8659969709057025, 'num_leaves': 159}. Best is trial 0 with value: -0.03713762915464922.
INFO:optuna.study.study:Trial 2 finished with value: -0.03730823215272649 and parameters: {'feature_fraction': 0.5780093202212182, 'num_leaves': 53}. Best is trial 0 with value: -0.03713762915464922.
INFO:optuna.study.study:Trial 3 finished with value: -0.03722653049641968 and parameters: {'feature_fraction': 0.5290418060840998, 'num_leaves': 223}. Best is trial 0 with value: -0.03713762915464922.
INFO:optuna.study.study:Trial 4 finished with value: -0.03715940620105788 and p

##### Получаем рекомендации

In [100]:
# отберем юзеров, по которым у нас есть информация из data train
users_lvl_2_val = pd.DataFrame(data_val_lvl_2['user_id'].unique())
users_lvl_2_val.columns = ['user_id']

train_users_lvl_2 = users_lvl_2['user_id'].unique()
users_lvl_2_val = users_lvl_2_val[users_lvl_2_val['user_id'].isin(train_users_lvl_2)]


# создаем dataframe, куда будем сохранять рекомендации
result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_2.columns=['user_id', 'actual']

result_lvl_2 = result_lvl_2[result_lvl_2['user_id'].isin(targets_lvl_2.user_id.unique()) ]

result_lvl_2 = users_lvl_2_val.merge(result_lvl_2, how='left', on='user_id')

In [101]:
# генерируем кандидатов
users_lvl_2_val['candidates'] = users_lvl_2_val['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=100))

In [102]:
# создаем dataframe для модели второго уровня с флагом купил / не купил
s = users_lvl_2_val.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

users_lvl_2_val = users_lvl_2_val.drop('candidates', axis=1).join(s)

targets_lvl_2_val = data_val_lvl_2[['user_id', 'item_id']].copy()

targets_lvl_2_val = users_lvl_2_val.merge(targets_lvl_2_val, on=['user_id', 'item_id'], how='left')

In [103]:
# добавим имеющиеся данные по пользователям и товарам
targets_lvl_2_val = targets_lvl_2_val.merge(item_features, on='item_id', how='left')
targets_lvl_2_val = targets_lvl_2_val.merge(user_features, on='user_id', how='left')

In [104]:
# генерируем новые признаки
# средний чек
average_receipt = data_val_lvl_2.groupby(['user_id', 'basket_id'])['sales_value'].sum().reset_index()
average_receipt = average_receipt.groupby('user_id')['sales_value'].mean().reset_index()
average_receipt.columns=['user_id', 'average_receipt']
targets_lvl_2_val = targets_lvl_2_val.merge(average_receipt, on='user_id', how='left')

In [105]:
# генерируем новые признаки
# кол-во покупок в каждой категории
purchases_per_category = data_val_lvl_2.merge(item_features, on='item_id', how='left')
purchases_per_category = purchases_per_category.groupby(['user_id', 'commodity_desc'])['quantity'].sum().reset_index()
purchases_per_category.columns=['user_id','commodity_desc', 'purchases_per_category']
targets_lvl_2_val = targets_lvl_2_val.merge(purchases_per_category, how='left', on=['user_id', 'commodity_desc'])

In [106]:
# генерируем новые признаки
# кол-во покупок в каждой под-категории
purchases_per_sub_category = data_val_lvl_2.merge(item_features, on='item_id', how='left')
purchases_per_sub_category = purchases_per_sub_category.groupby(['user_id', 'sub_commodity_desc'])['quantity'].sum().reset_index()
purchases_per_sub_category.columns=['user_id','sub_commodity_desc', 'purchases_per_sub_category']
targets_lvl_2_val = targets_lvl_2_val.merge(purchases_per_sub_category, how='left', on=['user_id', 'sub_commodity_desc'])

In [107]:
# генерируем новые признаки
# покупки в каждой под-категории в денежном эквиваленте
value_per_sub_category = data_val_lvl_2.merge(item_features, on='item_id', how='left')
value_per_sub_category = value_per_sub_category.groupby(['user_id', 'sub_commodity_desc'])['sales_value'].sum().reset_index()
value_per_sub_category.columns=['user_id','sub_commodity_desc', 'value_per_sub_category']
targets_lvl_2_val = targets_lvl_2_val.merge(value_per_sub_category, how='left', on=['user_id', 'sub_commodity_desc'])

In [108]:
# генерируем новые признаки
# среднее количество покупок в неделю для каждого товара
purchases_per_week = data_val_lvl_2.merge(item_features, on='item_id', how='left')
purchases_per_week = purchases_per_week.groupby(['item_id', 'week_no'])['quantity'].sum().reset_index()
purchases_per_week = purchases_per_week.groupby('item_id')['quantity'].mean().reset_index()
purchases_per_week.columns=['item_id', 'purchases_per_week']
targets_lvl_2_val = targets_lvl_2_val.merge(purchases_per_week, how='left', on='item_id')

In [109]:
# готовим данные
cat_feats = targets_lvl_2_val.columns[2:15].tolist()
targets_lvl_2_val[cat_feats] = targets_lvl_2_val[cat_feats].astype('category')

In [110]:
# LGBM
val_preds = lgb.predict_proba(targets_lvl_2_val)

In [111]:
# создаем dataframe с предсказаниями и считаем целевую метрику для модели первого уровня и модели 

val_preds = val_preds[:, 1]

result_val = targets_lvl_2_val
result_val['preds'] = val_preds

result_val = result_val.groupby(['user_id', 'item_id'])['preds'].mean().reset_index()
result_val = result_val.groupby('user_id').apply(lambda x: x.sort_values('preds', ascending=False).head(5).item_id.tolist())
result_val = result_lvl_2.merge(result_val.rename('lgbm'), how='left', on='user_id')

sorted(calc_metrics(result_val, precision_at_k, k=5), key=lambda precision_at_k: precision_at_k[1], reverse=True)

[('lgbm', 0.617373103087389)]

In [112]:
# Light AutoML
automl_pred_val = automl_model.predict(targets_lvl_2_val)

In [113]:
result = automl_pred_val.data
targets_lvl_2_val['preds_automl'] = result
targets_lvl_2_val['preds_automl'] = abs(targets_lvl_2_val['preds_automl'])

targets_lvl_2_val = targets_lvl_2_val.groupby(['user_id', 'item_id'])['preds_automl'].mean().reset_index()

result_val_automl = targets_lvl_2_val.groupby('user_id').apply(lambda x: x.sort_values('preds_automl', ascending=False).head(5).item_id.tolist())

result_val_automl = result_lvl_2.merge(result_val_automl.rename('automl'), how='left', on='user_id')

In [114]:
sorted(calc_metrics(result_val_automl, precision_at_k, k=5), key=lambda precision_at_k: precision_at_k[1], reverse=True)

[('automl', 0.6469911041339588)]

##### Добавляем рекомендации для новых пользователей

In [90]:
recs = pd.DataFrame(data_val_lvl_2['user_id'].unique())
recs.columns = ['user_id']

old_users = result_lvl_2['user_id'].unique()
new_users = recs[~recs['user_id'].isin(old_users)]

In [91]:
def popularity_recommendation(data, n=5):
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    return recs.tolist()

In [92]:
popular_recs = popularity_recommendation(data_train_lvl_1, n=6)
popular_recs = popular_recs[1:]

new_users['recs'] = new_users['user_id'].apply(lambda x: popular_recs)

/Users/nadejdaberzina/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


##### Объединяем все полученные рекомендации и проверяем качество модели

In [93]:
result_val_new = result_val_automl[['user_id', 'automl']]
result_val_new.rename(columns={'automl': 'recs'}, inplace=True)

/Users/nadejdaberzina/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [94]:
recommendations = pd.concat([result_val_new, new_users])

In [95]:
result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_2.columns=['user_id', 'actual']

In [96]:
recs = result_lvl_2.merge(recommendations, on='user_id', how='left')

In [97]:
sorted(calc_metrics(recs, precision_at_k, k=5), key=lambda precision_at_k: precision_at_k[1], reverse=True)

[('recs', 0.6143351458230328)]

Результат по precision@5, полученный на валидационном наборе, сопоставим с результатом, полученным на data_val_lvl_2.